# DVARS

In [ ]:
import os
import glob
import nibabel as nib
import numpy as np
from nipype.algorithms.confounds import ComputeDVARS 

# Path to the project directory
root_dir = "/data/project/HFSPain/satpads"

# Path to the directory with motion corrected data
dir = os.path.join(root_dir, "/data/project/HFSPain/satpads/derivatives/tsnr/")

# Empty list to store all DVARS values
all_dvars = []

inputs = [
    ("sub-DJL231215B", "cervical", "baseline"),
    ("sub-DJL231215B", "cervical", "satpads"),
    ("sub-DJL240109A", "cervical", "baseline"),
    ("sub-DJL240109A", "cervical", "satpads"),
    ("sub-DJL240111A", "cervical", "baseline"),
    ("sub-DJL240111A", "cervical", "satpads"),
    ("sub-DJL240124A", "cervical", "baseline"),
    ("sub-DJL240124A", "cervical", "satpads"),
    ("sub-DJL240125A", "cervical", "baseline"),
    ("sub-DJL240125A", "cervical", "satpads"),
    ("sub-DJL240207A", "cervical", "baseline"),
    ("sub-DJL240207A", "cervical", "satpads"),
    ("sub-DJL240208A", "cervical", "baseline"),
    ("sub-DJL240208A", "cervical", "satpads"),
    ("sub-DJL240220A", "cervical", "baseline"),
    ("sub-DJL240220A", "cervical", "satpads"),
    ("sub-DJL240221A", "cervical", "baseline"),
    ("sub-DJL240221A", "cervical", "satpads"),
    ("sub-DJL240307A", "cervical", "baseline"),
    ("sub-DJL240307A", "cervical", "satpads"),
    ("sub-DJL240219A", "lumbar", "baseline"),
    ("sub-DJL240219A", "lumbar", "satpads"),
    ("sub-DJL240305A", "lumbar", "baseline"),
    ("sub-DJL240305A", "lumbar", "satpads"),
    ("sub-DJL240410F", "lumbar", "baseline"),
    ("sub-DJL240410F", "lumbar", "satpads"),
    ("sub-DJL240424A", "lumbar", "baseline"),
    ("sub-DJL240424A", "lumbar", "satpads"),
    ("sub-DJL240425B", "lumbar", "baseline"),
    ("sub-DJL240425B", "lumbar", "satpads"),
    ("sub-DJL240425C", "lumbar", "baseline"),
    ("sub-DJL240425C", "lumbar", "satpads"),
    ("sub-DJL240425D", "lumbar", "baseline"),
    ("sub-DJL240425D", "lumbar", "satpads"),
    ("sub-DJL240521A", "lumbar", "baseline"),
    ("sub-DJL240521A", "lumbar", "satpads"),
    ("sub-DJL240521B", "lumbar", "baseline"),
    ("sub-DJL240521B", "lumbar", "satpads"),
    ("sub-DJL240521D", "lumbar", "baseline"),
    ("sub-DJL240521D", "lumbar", "satpads"),
]

for sub, segment, condition in inputs:
    data_path = glob.glob(os.path.join(dir, segment, sub, f"sub*{condition}*MCnofilt.nii.gz"))
    mask_path = glob.glob(os.path.join(dir, segment, sub, f"sub*{condition}*T2w_seg_reg.nii.gz"))

    dvars_interface = ComputeDVARS()
    dvars_interface.inputs.in_file = data_path[0]
    dvars_interface.inputs.in_mask = mask_path[0]
    dvars_interface.inputs.save_std = False
    
    res = dvars_interface.run()  # Run the DVARS computation

    # Store the results
    all_dvars.append((sub, segment, condition, res.outputs.avg_nstd, res.outputs.avg_std, res.outputs.avg_vxstd))

# Write all DVARS to a single file in the parent directory
with open(os.path.join(dir, "all_dvars_nipype.txt"), "w") as f:
    for sub, segment, condition, dvars_nstd, dvars_std, dvars_vxstd in all_dvars:
        f.write(f"{sub}, {segment}, {condition}, {dvars_nstd}, {dvars_std}, {dvars_vxstd}\n")

print(f"All DVARS values have been written to {os.path.join(dir, 'all_dvars_nipype.txt')}")

# FD

In [ ]:
import os
import glob
import numpy as np

def calculate_mean_fd_from_slices(motion_files_pattern):
    """
    Calculate mean volume framewise displacement from slicewise motion parameter files.
    
    Parameters:
    motion_files_pattern: str - glob pattern to find all slicewise motion parameter files
    
    Returns:
    float - mean framewise displacement
    array - framewise displacement timeseries
    """
    # Find all slicewise motion parameter files
    motion_files = glob.glob(motion_files_pattern)
    
    if not motion_files:
        raise ValueError(f"No motion files found for pattern: {motion_files_pattern}")
    
    print(f"Found {len(motion_files)} slicewise motion files")
    
    # Load motion parameters from all slices
    all_x_params = []
    all_y_params = []
    
    for motion_file in sorted(motion_files):  # Sort to ensure consistent slice order
        try:
            motion_data = np.loadtxt(motion_file, skiprows=3) # Skipping header
            
            # Extract x and y translation columns
            x_params = motion_data[:, 0]  # First column = x translation
            y_params = motion_data[:, 1]  # Second column = y translation
            
            all_x_params.append(x_params)
            all_y_params.append(y_params)
            
        except Exception as e:
            print(f"Error loading {motion_file}: {e}")
            continue
    
    if not all_x_params:
        raise ValueError("No valid motion parameter files found")
    
    # Convert to numpy arrays
    all_x_params = np.array(all_x_params)  # Shape: [n_slices, n_timepoints]
    all_y_params = np.array(all_y_params)  # Shape: [n_slices, n_timepoints]
    
    # Average across slices to get volume-level motion parameters
    avg_x_translation = np.mean(all_x_params, axis=0)  # Shape: [n_timepoints]
    avg_y_translation = np.mean(all_y_params, axis=0)  # Shape: [n_timepoints]

    # Calculate framewise displacement
    # FD = sqrt((dx)^2 + (dy)^2) for each timepoint
    dx = np.diff(avg_x_translation)  # Frame-to-frame differences
    dy = np.diff(avg_y_translation)  # Frame-to-frame differences

    fd_timeseries = np.sqrt(dx**2 + dy**2)
    mean_fd = np.mean(fd_timeseries)

    return mean_fd, fd_timeseries, avg_x_translation, avg_y_translation

# Path to the project directory
root_dir = "/data/project/HFSPain/satpads"

# Path to the directory with motion corrected data
dir = os.path.join(root_dir, "/data/project/HFSPain/satpads/derivatives/tsnr/")

# Empty list to store all FD results
all_mean_fd = []

inputs = [
    ("sub-DJL231215B", "cervical", "baseline"),
    ("sub-DJL231215B", "cervical", "satpads"),
    ("sub-DJL240109A", "cervical", "baseline"),
    ("sub-DJL240109A", "cervical", "satpads"),
    ("sub-DJL240111A", "cervical", "baseline"),
    ("sub-DJL240111A", "cervical", "satpads"),
    ("sub-DJL240124A", "cervical", "baseline"),
    ("sub-DJL240124A", "cervical", "satpads"),
    ("sub-DJL240125A", "cervical", "baseline"),
    ("sub-DJL240125A", "cervical", "satpads"),
    ("sub-DJL240207A", "cervical", "baseline"),
    ("sub-DJL240207A", "cervical", "satpads"),
    ("sub-DJL240208A", "cervical", "baseline"),
    ("sub-DJL240208A", "cervical", "satpads"),
    ("sub-DJL240220A", "cervical", "baseline"),
    ("sub-DJL240220A", "cervical", "satpads"),
    ("sub-DJL240221A", "cervical", "baseline"),
    ("sub-DJL240221A", "cervical", "satpads"),
    ("sub-DJL240307A", "cervical", "baseline"),
    ("sub-DJL240307A", "cervical", "satpads"),
    ("sub-DJL240219A", "lumbar", "baseline"),
    ("sub-DJL240219A", "lumbar", "satpads"),
    ("sub-DJL240305A", "lumbar", "baseline"),
    ("sub-DJL240305A", "lumbar", "satpads"),
    ("sub-DJL240410F", "lumbar", "baseline"),
    ("sub-DJL240410F", "lumbar", "satpads"),
    ("sub-DJL240424A", "lumbar", "baseline"),
    ("sub-DJL240424A", "lumbar", "satpads"),
    ("sub-DJL240425B", "lumbar", "baseline"),
    ("sub-DJL240425B", "lumbar", "satpads"),
    ("sub-DJL240425C", "lumbar", "baseline"),
    ("sub-DJL240425C", "lumbar", "satpads"),
    ("sub-DJL240425D", "lumbar", "baseline"),
    ("sub-DJL240425D", "lumbar", "satpads"),
    ("sub-DJL240521A", "lumbar", "baseline"),
    ("sub-DJL240521A", "lumbar", "satpads"),
    ("sub-DJL240521B", "lumbar", "baseline"),
    ("sub-DJL240521B", "lumbar", "satpads"),
    ("sub-DJL240521D", "lumbar", "baseline"),
    ("sub-DJL240521D", "lumbar", "satpads"),
]

for sub, segment, condition in inputs:
    print(f"Processing FD for {sub}, {segment}, {condition}")
    
    moco_file = os.path.join(dir, segment, sub, f"moco_params_{condition}", "slice*MC_params.txt")

    mean_fd, fd_timeseries, avg_x, avg_y = calculate_mean_fd_from_slices(moco_file)
    
    # Store the results
    all_mean_fd.append((sub, segment, condition, mean_fd))

# Write all FD to a single file in the parent directory
with open(os.path.join(dir, "all_fd_updated.txt"), "w") as f:
    for sub, segment, condition, mean_fd in all_mean_fd:
        f.write(f"{sub}, {segment}, {condition}, {mean_fd}\n")

print(f"All FD values have been written to {os.path.join(dir, 'all_fd_updated.txt')}")